In [1]:
import os
from pathlib import Path

import pytesseract
from pdf2image import convert_from_path
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
import os

load_dotenv()


print("Tesseract version from Python:", pytesseract.get_tesseract_version())

/Users/soul./Desktop/Rahul/Natural Language Processing/Belleville By-Law Bot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tesseract version from Python: 5.5.1


In [2]:
DATA_DIR = Path("data")
RAW_PDF_DIR = DATA_DIR / "raw_pdfs"
OCR_JSON_DIR = DATA_DIR / "ocr_json"

OCR_JSON_DIR.mkdir(parents=True, exist_ok=True)
print("OCR output folder:", OCR_JSON_DIR)


OCR output folder: data/ocr_json


In [3]:
def ocr_pdf_to_json(pdf_path: Path, output_path: Path):
    """
    OCR a scanned PDF into a JSON structure:
    {
      "file_name": "...",
      "pages": [
        {"page_number": 1, "text": "..."},
        {"page_number": 2, "text": "..."},
        ...
      ]
    }
    """
    print(f"Starting OCR for: {pdf_path.name}")
    
    # Convert all pages to images
    pages = convert_from_path(str(pdf_path))
    print(f"Total pages: {len(pages)}")
    
    ocr_pages = []
    
    for i, img in enumerate(pages, start=1):
        print(f"  - OCR on page {i}...")
        text = pytesseract.image_to_string(img, lang="eng")
        ocr_pages.append({
            "page_number": i,
            "text": text
        })
    
    data = {
        "file_name": pdf_path.name,
        "pages": ocr_pages
    }
    
    # Save to JSON
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    print(f"OCR complete! Saved to {output_path}")

In [4]:
# def ocr_pdf_to_json(pdf_path: Path, output_path: Path):
#     """
#     OCR a scanned PDF into a JSON structure:
#       { file_name, pages: [{page_number, text, error?}] }
#     """
#     print(f"Starting OCR for: {pdf_path.name}")

#     # Higher DPI for clearer images
#     pages = convert_from_path(str(pdf_path), dpi=300)
#     print(f"Total pages: {len(pages)}")

#     ocr_pages = []

#     for i, img in enumerate(pages, start=1):
#         print(f"  - OCR on page {i}...")
#         try:
#             # PSM 6 = assume a block of text; tweak if layout differs
#             text = pytesseract.image_to_string(img, lang="eng", config="--psm 6")
#             ocr_pages.append({
#                 "page_number": i,
#                 "text": text  # raw text already stored
#             })
#         except Exception as e:
#             print(f"    OCR error on page {i}: {e}")
#             ocr_pages.append({
#                 "page_number": i,
#                 "text": "",
#                 "error": str(e)  # keep per-page error info
#             })

#     data = {
#         "file_name": pdf_path.name,
#         "pages": ocr_pages
#     }

#     with open(output_path, "w", encoding="utf-8") as f:
#         json.dump(data, f, ensure_ascii=False, indent=2)

#     print(f"OCR complete! Saved to {output_path}")


In [5]:
from pathlib import Path

# Just to be sure these are defined
DATA_DIR = Path("data")
RAW_PDF_DIR = DATA_DIR / "raw_pdfs"
OCR_JSON_DIR = DATA_DIR / "ocr_json"
OCR_JSON_DIR.mkdir(parents=True, exist_ok=True)

for pdf_path in RAW_PDF_DIR.glob("*.pdf"):
    # Create a cleaner JSON file name from the PDF name
    output_name = pdf_path.stem.replace(" ", "_") + ".json"
    output_path = OCR_JSON_DIR / output_name
    
    # Skip if we already have OCR for this file
    if output_path.exists():
        print(f"Skipping (already exists): {output_path.name}")
        continue
    
    print(f"\nProcessing PDF: {pdf_path.name}")
    ocr_pdf_to_json(pdf_path, output_path)


Skipping (already exists): 2025_Fees___Charges_-_BL_2024-201.json
Skipping (already exists): BL_2001-129_A_Bylaw_Respecting_Streets_and_Driveway_Controls.json
Skipping (already exists): BL_0098-175_A_Bylaw_for_Establishing_Waste_Collection_and_Fees.json


In [6]:
OCR_JSON_DIR = DATA_DIR / "ocr_json"
CHUNKS_JSON_PATH = DATA_DIR / "bylaw_chunks.json"

print("OCR JSON directory:", OCR_JSON_DIR)
print("Chunks will be saved to:", CHUNKS_JSON_PATH)

OCR JSON directory: data/ocr_json
Chunks will be saved to: data/bylaw_chunks.json


In [7]:
def load_ocr_json_files(ocr_dir: Path):
    docs = []
    for json_path in ocr_dir.glob("*.json"):
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        docs.append(data)
    print(f"Loaded {len(docs)} OCR JSON files.")
    return docs

ocr_docs = load_ocr_json_files(OCR_JSON_DIR)
len(ocr_docs)

Loaded 3 OCR JSON files.


3

In [8]:
def clean_text(text: str) -> str:
    # Replace newlines with spaces
    text = text.replace("\n", " ")
    # Collapse multiple spaces into one
    text = " ".join(text.split())
    return text

In [9]:

def iter_page_chunks_from_document(doc, chunk_size=800, overlap=200):
    """
    doc: dict loaded from one OCR JSON ({file_name, pages: [...]})
    Yields small chunks per page.
    """
    file_name = doc["file_name"]
    bylaw_name = Path(file_name).stem.replace(" ", "_")

    for page in doc["pages"]:
        page_num = page["page_number"]
        page_text = clean_text(page["text"])

        if not page_text:
            continue  # skip empty pages

        text_len = len(page_text)
        if text_len <= chunk_size:
            # Just one chunk for this page
            yield {
                "id": f"{bylaw_name}_p{page_num}_chunk_0",
                "file_name": file_name,
                "bylaw_name": bylaw_name,
                "page_number": page_num,
                "text": page_text,
            }
        else:
            # Split long page into multiple chunks
            start = 0
            chunk_id = 0

            # Safety: make sure chunk_size > overlap so we don't loop forever
            if overlap >= chunk_size:
                raise ValueError("overlap must be smaller than chunk_size")

            while start < text_len:
                end = min(start + chunk_size, text_len)
                chunk_text = page_text[start:end]

                yield {
                    "id": f"{bylaw_name}_p{page_num}_chunk_{chunk_id}",
                    "file_name": file_name,
                    "bylaw_name": bylaw_name,
                    "page_number": page_num,
                    "text": chunk_text,
                }

                chunk_id += 1
                if end == text_len:
                    break  # reached end of page text
                start = end - overlap  # move with overlap and continue


In [10]:
# Build and save chunks from OCR documents
chunks = []
for doc in ocr_docs:
    chunks.extend(iter_page_chunks_from_document(doc, chunk_size=800, overlap=200))

print(f"Generated {len(chunks)} chunks")
CHUNKS_JSON_PATH.parent.mkdir(parents=True, exist_ok=True)
with open(CHUNKS_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)
print(f"Saved chunks to {CHUNKS_JSON_PATH}")


Generated 368 chunks
Saved chunks to data/bylaw_chunks.json


In [11]:
with open(CHUNKS_JSON_PATH, "r", encoding="utf-8") as f:
    chunks = json.load(f)

print("Number of chunks loaded:", len(chunks))
print("Example keys:", chunks[0].keys())
print("Sample text:", chunks[0]["text"][:200], "...")

Number of chunks loaded: 368
Example keys: dict_keys(['id', 'file_name', 'bylaw_name', 'page_number', 'text'])
Sample text: THE CORPORATION OF THE CITY OF BELLEVILLE BY-LAW NUMBER 2001-129 A BY-LAW RESPECTING STREETS AND DRIVEWAY CONTROLS WHEREAS Section 308 of the Municipal Act R.S.O. 1990, c. M45, as amended provides tha ...


In [12]:
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

embed_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
print("Loaded embedding model:", EMBEDDING_MODEL_NAME)

Loaded embedding model: sentence-transformers/all-MiniLM-L6-v2


In [13]:
# Extract texts from chunks
texts = [c["text"] for c in chunks]
print("Total texts to embed:", len(texts))

# Create embeddings
embeddings = embed_model.encode(
    texts,
    show_progress_bar=True,
    convert_to_numpy=True
)

print("Embeddings shape before cast:", embeddings.shape, embeddings.dtype)

# FAISS prefers float32
embeddings = embeddings.astype("float32")

# Normalize for cosine similarity
faiss.normalize_L2(embeddings)

print("Embeddings ready. Shape:", embeddings.shape, embeddings.dtype)

Total texts to embed: 368


Batches: 100%|██████████| 12/12 [00:01<00:00,  7.81it/s]

Embeddings shape before cast: (368, 384) float32
Embeddings ready. Shape: (368, 384) float32


In [14]:
VECTOR_DIM = embeddings.shape[1]
index = faiss.IndexFlatIP(VECTOR_DIM)  # IP = inner product
index.add(embeddings)

print("FAISS index size:", index.ntotal)

FAISS index size: 368


In [15]:
INDEX_PATH = DATA_DIR / "bylaw_faiss.index"
METADATA_PATH = DATA_DIR / "bylaw_metadata.json"

# Save FAISS index
faiss.write_index(index, str(INDEX_PATH))
print("Saved FAISS index to:", INDEX_PATH)

# Build & save metadata (everything except embeddings)
metadata = [
    {
        "id": c["id"],
        "file_name": c["file_name"],
        "bylaw_name": c.get("bylaw_name"),
        "page_number": c.get("page_number"),
        "text": c["text"],
    }
    for c in chunks
]

with open(METADATA_PATH, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print("Saved metadata to:", METADATA_PATH)

Saved FAISS index to: data/bylaw_faiss.index
Saved metadata to: data/bylaw_metadata.json


In [16]:
index = faiss.read_index(str(INDEX_PATH))

with open(METADATA_PATH, "r", encoding="utf-8") as f:
    metadata = json.load(f)

In [17]:
def retrieve_top_k(query: str, k: int = 5):
    # 1. Embed the query
    q_emb = embed_model.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)

    # 2. Search FAISS index
    D, I = index.search(q_emb, k)
    
    # 3. Collect results
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1:
            continue
        chunk_info = metadata[idx]
        results.append({
            "score": float(score),
            "id": chunk_info["id"],
            "file_name": chunk_info["file_name"],
            "bylaw_name": chunk_info.get("bylaw_name"),
            "page_number": chunk_info.get("page_number"),
            "text": chunk_info["text"],
        })
    return results

In [18]:
query = "What fees and charges does the city collect?"
results = retrieve_top_k(query, k=3)

print("Query:", query)
for i, r in enumerate(results, start=1):
    print(f"\nResult {i}:")
    print("Score:", round(r["score"], 3))
    print("File:", r["file_name"])
    print("Bylaw:", r["bylaw_name"])
    print("Page:", r["page_number"])
    print("Text sample:", r["text"][:300], "...")

Query: What fees and charges does the city collect?

Result 1:
Score: 0.61
File: 2025 Fees _ Charges - BL 2024-201.pdf
Bylaw: 2025_Fees___Charges_-_BL_2024-201
Page: 10
Text sample: Change of Ownership — (Property Taxes $25 $30 Change of Occupancy/New Account — Water Bill} New Account $50 $50 Transfer to Taxes — per transaction $20 $20 Payment Transfer (wrong account number) $30 $30 Refund/Overpayment recovery (Tax Bill, Water $30 $35 Bill) Title Search — Property in second yea ...

Result 2:
Score: 0.509
File: 2025 Fees _ Charges - BL 2024-201.pdf
Bylaw: 2025_Fees___Charges_-_BL_2024-201
Page: 10
Text sample: arges are authorized under this by-law. ccept credit card Multi Property Billing List $2 per roll $3 per roll Post Date Cheque Return $15 $15 Corporate Search Recovery Charge $20 $25 Tax Sale costs Actual Recoverable Cost WATER RATES Actual Recoverable Cost Metered First 455 cubic metres $1.99 $2.02 ...

Result 3:
Score: 0.506
File: 2025 Fees _ Charges - BL 2024-201.pdf
Bylaw: 20

In [19]:
HF_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN")
print("HF token found?", HF_TOKEN is not None)

HF token found? True


In [20]:
HF_LLM_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

client = InferenceClient(
    model=HF_LLM_MODEL_NAME,
    token=HF_TOKEN,
)


In [21]:
def build_context_from_results(results):
    """
    results: list of dicts from retrieve_top_k
    Returns: a single string with sources separated by --- markers.
    """
    parts = []
    for r in results:
        header = f"[{r.get('bylaw_name')} | page {r.get('page_number')} | {r.get('file_name')}]"
        parts.append(header + "\n" + r["text"])
    return "\n\n---\n\n".join(parts)

In [22]:
import ollama

OLLAMA_MODEL_NAME = "llama3" 

In [23]:
def call_llm_with_context(question: str, context: str, max_new_tokens: int = 300) -> str:
    system_prompt = (
        "You are an assistant that answers questions about City of Belleville by-laws.\n"
        "Use ONLY the information in the provided CONTEXT.\n"
        "If the answer is not clearly in the context, say you are not sure and suggest "
        "contacting the City of Belleville for confirmation.\n\n"
        "Format your answer as:\n"
        "- First, a short 1–2 sentence summary in plain language.\n"
        "- Then, a bullet list focusing ONLY on fees or dollar amounts relevant to the question.\n"
        "- Each bullet must be of the form: '<item or waste type>: $<amount> (short description if needed)'.\n"
        "- Do NOT include operational rules (like number of lifts, tag requirements, schedule rules) in the bullet list;\n"
        "  those can be mentioned in the summary if important.\n"
        "- Do NOT leave any bullet unfinished. Do NOT end the answer with a hanging '-' or incomplete sentence.\n"
        "Do NOT invent information that is not supported by the context.\n"
        "Do NOT ask or answer extra questions. Just answer the user's question.\n"  
    )

    user_message = (
        f"Here is the context from Belleville by-laws:\n\n"
        f"{context}\n\n"
        f"Based ONLY on the context above, answer this question clearly and concisely:\n"
        f"{question}"
    )

    # response = client.chat_completion(
    #     model=HF_LLM_MODEL_NAME,
    #     messages=[
    #         {"role": "system", "content": system_prompt},
    #         {"role": "user", "content": user_message},
    #     ],
    #     max_tokens=max_new_tokens,
    #     temperature=0.2,
    # )

    response = ollama.chat(
        model = "llama3",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message},
        ],
        options={
            "num_predict": max_new_tokens,
            "temperature": 0.2,
        }
    )

    # msg = response.choices[0].message
    content = response["message"]["content"]
    # content = msg["content"] if isinstance(msg, dict) else msg.content
    return content.strip()

In [24]:
def answer_question_with_rag(question: str, k: int = 5):
    q_lower = question.lower().strip()

    # 1. Handle greetings / capability questions explicitly
    greeting_phrases = [
        "hello", "hi", "hey",
        "what can you do", "who are you",
        "help", "how can you help me",
    ]

    if any(p in q_lower for p in greeting_phrases):
        intro_answer = (
            "Hi! I’m the Belleville By-Law Assistant.\n\n"
            "- I can help you understand City of Belleville by-laws in plain language.\n"
            "- You can ask about things like parking rules, noise restrictions, property standards,\n"
            "  fees and charges, animals/pets, streets and driveways, and other municipal rules.\n"
            "- I’ll look up the relevant by-law sections and summarize them for you, and I’ll tell you\n"
            "  which by-law and page the information comes from.\n\n"
            "Try asking something like:\n"
            "- \"Can I park on the street overnight in Belleville?\"\n"
            "- \"What are the fees for a change of ownership?\"\n"
            "- \"Are there any noise restrictions at night?\""
        )
        return {
            "answer": intro_answer,
            "sources": [],
        }

    # 2. Normal RAG flow: retrieve top-k relevant chunks
    results = retrieve_top_k(question, k=k)
    if not results:
        return {
            "answer": (
                "I couldn’t find any relevant sections in the by-laws for that question. "
                "Try asking specifically about parking, noise, fees, property standards, "
                "streets/driveways, or other Belleville by-laws."
            ),
            "sources": [],
        }

    # Check top similarity score to see if this question is relevant at all
    top_score = results[0]["score"]
    if top_score < 0.35:  # you can tune this threshold
        return {
            "answer": (
                "I checked the by-laws I have, but nothing seems clearly related to that question.\n\n"
                "I’m best at answering questions specifically about City of Belleville by-laws — "
                "for example parking rules, noise by-laws, fees and charges, or property-related rules."
            ),
            "sources": [],
        }

    # 3. Build context string from the retrieved chunks
    context = build_context_from_results(results)
    
    # 4. Get LLM answer (summary + bullet list)
    main_answer = call_llm_with_context(question, context)
    
    # 5. Build structured source info from results
    sources = [
        {
            "file_name": r["file_name"],
            "bylaw_name": r["bylaw_name"],
            "page_number": r["page_number"],
            "score": r["score"],
        }
        for r in results
    ]

    # 6. Build final source line
    seen = set()
    source_bits = []
    for s in sources:
        key = (s["bylaw_name"], s["page_number"])
        if key in seen:
            continue
        seen.add(key)
        if s["bylaw_name"] is not None and s["page_number"] is not None:
            source_bits.append(f"{s['bylaw_name']} (page {s['page_number']})")

    if source_bits:
        source_line = "_Source by-laws: " + ", ".join(source_bits) + "._"
        full_answer = main_answer + "\n\n" + source_line
    else:
        full_answer = main_answer

    return {
        "answer": full_answer,
        "sources": sources,
    }

In [25]:
def chat_loop():
    print("Belleville By-Law Assistant")
    print("Type your question about by-laws.")
    print("Type 'exit' or 'quit' to stop.\n")

    while True:
        question = input("You: ").strip()
        if question.lower() in ("exit", "quit"):
            print("Bot: Goodbye!")
            break

        if not question:
            continue  # skip empty input

        res = answer_question_with_rag(question, k=5)
        print("\nYou:", question)
        print("\nBot:", res["answer"])
        print("\nSources:")
        for s in res["sources"]:
            print(
                f"- {s['file_name']} | {s['bylaw_name']} | "
                f"page {s['page_number']} | score={s['score']:.3f}"
            )
        print("\n" + "-" * 60 + "\n")

In [26]:
# chat_loop()

In [27]:
def rag_chat(message, history):
    result = answer_question_with_rag(message, k=5)
    answer = result["answer"]

    # Build a simple sources text block
    if result["sources"]:
        src_lines = ["\n\n**Sources:**"]
        for s in result["sources"]:
            src_lines.append(
                f"- `{s['bylaw_name']}` (page {s['page_number']}, score={s['score']:.3f})"
            )
        answer = answer + "\n" + "\n".join(src_lines)

    return answer

In [28]:
import gradio as gr

demo = gr.ChatInterface(
    fn=rag_chat,
    title="⚖️ Belleville By-Law Assistant",
    description=(
        "Ask questions about City of Belleville by-laws (parking, noise, waste, fees, "
        "property standards, streets/driveways, etc.).\n\n"
        "I will search the by-laws and summarize the relevant sections for you."
    ),
    examples=[
        "Can I park on the street overnight in Belleville?",
        "What are the waste collection fees?",
        "What fees and charges does the city collect?",
        "Are there any noise restrictions at night?",
    ],
)

demo.launch()

/Users/soul./Desktop/Rahul/Natural Language Processing/Belleville By-Law Bot/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
